In [45]:
import numpy as np

from strat import RSIRisingFalling
from quantfreedom.helper_funcs import dos_cart_product, get_dos, log_dynamic_order_settings
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
candles = dl_ex_candles(
    exchange="bybit",
    symbol="BTCUSDT",
    timeframe="1m",
    candles_to_dl=6000,
)

In [12]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=0,
)
short_strat.plot_signals(candles=candles)

In [13]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35, 61, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [14]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [15]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 6,000
Total candles to test: 1,728,000


In [16]:
backtest_results.sort_values(by=["total_pnl"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
60,1,12,39.0,18,21,136.077,46.154,0.688,834.968,2360.771,1360.771
93,1,45,17.0,7,10,136.040,41.176,0.413,242.246,2360.401,1360.401
86,1,38,31.0,11,20,132.716,35.484,0.533,793.590,2327.155,1327.155
84,1,36,34.0,13,21,118.504,38.235,0.619,849.391,2185.042,1185.042
62,1,14,36.0,15,21,110.349,41.667,0.655,754.848,2103.494,1103.494
39,0,39,21.0,7,14,107.245,33.333,0.539,348.566,2072.451,1072.451
37,0,37,21.0,7,14,105.397,33.333,0.519,352.326,2053.968,1053.968
61,1,13,36.0,15,21,100.229,41.667,0.655,745.507,2002.290,1002.290
63,1,15,36.0,15,21,100.229,41.667,0.655,745.507,2002.290,1002.290
68,1,20,26.0,14,12,96.732,53.846,0.759,376.753,1967.319,967.319


In [18]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    dos_index=12,
    ind_set_index=1,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.0025,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.01,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 40.0,
    rsi_length = 14,
)


c:\Users\kcies\coding\venv\qfree\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [38]:
dos_cart_arrays = dos_cart_product(dos_arrays=dos_arrays)


In [40]:
dynamic_order_settings = get_dos(
    dos_cart_arrays=dos_cart_arrays,
    dos_index=12,
)

dynamic_order_settings

DynamicOrderSettings(max_equity_risk_pct=0.12, max_trades=5, risk_account_pct_size=0.03, risk_reward=2.0, sl_based_on_add_pct=0.0025, sl_based_on_lookback=50, sl_bcb_type=3, sl_to_be_cb_type=6, sl_to_be_when_pct=0.0, trail_sl_bcb_type=3, trail_sl_by_pct=0.005, trail_sl_when_pct=0.01)

In [25]:
log_dynamic_order_settings(dos_index= 12, dynamic_order_settings= dynamic_order_settings)

In [28]:
a = dynamic_order_settings
a

DynamicOrderSettings(max_equity_risk_pct=0.12, max_trades=5, risk_account_pct_size=0.03, risk_reward=2.0, sl_based_on_add_pct=0.0025, sl_based_on_lookback=50, sl_bcb_type=3, sl_to_be_cb_type=6, sl_to_be_when_pct=0.0, trail_sl_bcb_type=3, trail_sl_by_pct=0.005, trail_sl_when_pct=0.01)

In [33]:
backtest_results_fixed_dos = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dynamic_order_settings,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 1
Total combinations of settings to test: 6
Total candles: 6,000
Total candles to test: 36,000
